In [1]:
import os
import sys

import torch
from torch.utils.tensorboard import SummaryWriter

############################################
# Added for GluNet package
############################################
from trainer_glunet import LatentODEWrapper
from eval_glunet import test
sys.path.insert(1, '../..')
os.chdir('../..')
# utils for darts
from utils.darts_dataset import *
from utils.darts_processing import *

In [2]:
# define data
formatter, series, scalers = load_data(seed=0, 
                                       study_file=None, 
                                       dataset='colas', 
                                       use_covs=True, 
                                       cov_type='dual',
                                       use_static_covs=True)

dataset_train = SamplingDatasetDual(series['train']['target'],
                                    series['train']['future'],
                                    output_chunk_length=12,
                                    input_chunk_length=48,
                                    use_static_covariates=True,
                                    max_samples_per_ts=100,
                                    )
dataset_val = SamplingDatasetDual(series['val']['target'],
                                    series['val']['future'],   
                                    output_chunk_length=12,
                                    input_chunk_length=48,
                                    use_static_covariates=True,)
dataset_test = SamplingDatasetInferenceDual(target_series=series['test']['target'],
                                            covariates=series['test']['future'],
                                            input_chunk_length=48,
                                            output_chunk_length=12,
                                            use_static_covariates=True,
                                            array_output_only=True)


--------------------------------
Loading column definition...
Checking column definition...
Loading data...
Dropping columns / rows...
Checking for NA values...
Setting data types...
Dropping columns / rows...
Encoding data...
	Updated column definition:
		id: REAL_VALUED (ID)
		time: DATE (TIME)
		gl: REAL_VALUED (TARGET)
		gender: REAL_VALUED (STATIC_INPUT)
		age: REAL_VALUED (STATIC_INPUT)
		BMI: REAL_VALUED (STATIC_INPUT)
		glycaemia: REAL_VALUED (STATIC_INPUT)
		HbA1c: REAL_VALUED (STATIC_INPUT)
		follow.up: REAL_VALUED (STATIC_INPUT)
		T2DM: REAL_VALUED (STATIC_INPUT)
		time_year: REAL_VALUED (KNOWN_INPUT)
		time_month: REAL_VALUED (KNOWN_INPUT)
		time_day: REAL_VALUED (KNOWN_INPUT)
		time_hour: REAL_VALUED (KNOWN_INPUT)
		time_minute: REAL_VALUED (KNOWN_INPUT)
Interpolating data...
	Dropped segments: 63
	Extracted segments: 205
	Interpolated values: 241
	Percent of values interpolated: 0.22%
Splitting data...
	Train: 72275 (45.89%)
	Val: 35713 (22.68%)
	Test: 38253 (24.29%)
	Tes

In [3]:
model_wrapper = LatentODEWrapper(device = 'cuda')

In [ ]:
# get torch summary writer and initialize at the directory: ./output/tensorboard_latentode_colas/
logger = SummaryWriter(log_dir='./output/tensorboard_latentode_colas/')

model_wrapper.fit(train_dataset=dataset_train,
                    val_dataset=dataset_val,
                    learning_rate= 1e-3,
                    batch_size= 128,
                    epochs= 2,
                    num_samples= 10,
                    device= 'cuda',
                    model_path='./output/tensorboard_latentode_colas/model.ckpt',
                    trial= None,
                    logger= logger)

/home/grads/m/mrsergazinov/.conda/envs/glunet/lib/python3.10/site-packages/torchdiffeq/_impl/misc.py:296: UserWarning: t is not on the same device as y0. Coercing to y0.device.
  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")


In [4]:
model_wrapper.load(model_path='./output/tensorboard_latentode_colas/model.ckpt',
                    device= 'cuda')

In [5]:
predictions = model_wrapper.predict(dataset_test,
                                    num_samples=10,
                                    batch_size=128,
                                    device='cuda')
trues = np.array([dataset_test.evalsample(i).values() for i in range(len(dataset_test))])
trues = (trues - scalers['target'].min_) / scalers['target'].scale_
obsrv_std = 0.01 / scalers['target'].scale_
predictions = (predictions - scalers['target'].min_) / scalers['target'].scale_

/home/grads/m/mrsergazinov/.conda/envs/glunet/lib/python3.10/site-packages/torchdiffeq/_impl/misc.py:296: UserWarning: t is not on the same device as y0. Coercing to y0.device.
  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")


In [10]:
errors, _, _ = test(trues, predictions, obsrv_std)

In [11]:
np.median(errors, axis=0)

array([65.999405,  7.235167], dtype=float32)

# Legacy (below)

In [5]:
import torch
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=128, shuffle=False)
batch = iter(dataloader_val).next()
device = 'cuda'
inp_len, out_len = 48, 12
batch_dict = {
            'observed_data': batch[0].to(device),
            'observed_tp': torch.arange(0, inp_len).to(device) / 12,
            'data_to_predict': batch[-1].to(device),
            'tp_to_predict': torch.arange(inp_len, inp_len+out_len).to(device) / 12,
            'observed_mask': torch.ones(batch[0].shape).to(device),
            'mask_predicted_data': None,
            'labels': None,
            'mode': 'extrap',
	    }
pred_y, info = model_wrapper.model.get_reconstruction(batch_dict["tp_to_predict"], 
                                                        batch_dict["observed_data"], 
                                                        batch_dict["observed_tp"], 
                                                        mask = batch_dict["observed_mask"], 
                                                        n_traj_samples = 100,
                                                        mode = batch_dict["mode"])


/home/grads/m/mrsergazinov/.conda/envs/glunet/lib/python3.10/site-packages/torchdiffeq/_impl/misc.py:296: UserWarning: t is not on the same device as y0. Coercing to y0.device.
  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")


In [7]:
pred_y = pred_y.detach().cpu().numpy()
y = batch_dict['data_to_predict'].detach().cpu().numpy()

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4, 4, figsize=(40, 20))
for i in range(16):
    ax = axs[i // 4, i % 4]
    df = pd.DataFrame(pred_y[:, i, :, 0].transpose(1, 0))
    df = pd.melt(df.reset_index(), id_vars=['index'], value_vars=df.columns)
    df['type'] = 'pred'
    df2 = pd.DataFrame(y[i, :, 0])
    df2 = pd.melt(df2.reset_index(), id_vars=['index'], value_vars=df2.columns)
    df2['type'] = 'true'
    df = pd.concat([df, df2])
    sns.lineplot(x="index", y="value", hue='type', data=df, ax=ax)
fig.savefig("pred_vs_true.png")

In [4]:
# define params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 1
classif_per_tp = False
n_labels = 1
class args_parser:
    z0_encoder = 'odernn'
    latents = 10
    rec_dims = 20
    rec_layers = 1
    gen_layers = 1
    units = 100
    gru_units = 100
    
    extrap = True    
    poisson = False
    classif = False
    linear_classif = False
    dataset = 'glunet'
args = args_parser()

# define model
obsrv_std = 0.01
obsrv_std = torch.Tensor([obsrv_std]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))
model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device, 
                                classif_per_tp = classif_per_tp,
                                n_labels = n_labels,)

In [16]:
# log_path = "logs/" + file_name + "_" + str(experimentID) + ".log"
# if not os.path.exists("logs/"):
# 	utils.makedirs("logs/")
log_path = './output/latentode_colas.log'
logger = latentode_utils.get_logger(logpath=log_path, filepath='./')

optimizer = optim.Adamax(model.parameters(), lr=1e-3)

num_batches = 100
niters = 100

for itr in range(1, num_batches * (niters + 1)):
	optimizer.zero_grad()
	latentode_utils.update_learning_rate(optimizer, decay_rate = 0.999, lowest = 1e-3 / 10)

	wait_until_kl_inc = 10
	if itr // num_batches < wait_until_kl_inc:
		kl_coef = 0.
	else:
		kl_coef = (1-0.99** (itr // num_batches - wait_until_kl_inc))

	batch = train_loader.__next__()
	inp_len, out_len = batch[0].shape[0], batch[0].shape[1], batch[-1].shape[1]
	batch_dict = {
		'observed_data': batch[0].to(device),
        'observed_tp': torch.arange(0, inp_len).to(device) / 12,
        'data_to_predict': batch[-1].to(device),
        'tp_to_predict': torch.arange(inp_len, inp_len+out_len).to(device) / 12,
        'observed_mask': torch.ones(batch[0].shape).to(device),
        'mask_predicted_data': None,
        'labels': None,
        'mode': 'extrap'
	}
	train_res = model.compute_all_losses(batch_dict, n_traj_samples = 3, kl_coef = kl_coef)
	train_res["loss"].backward()
	optimizer.step()

	if itr % num_batches == 0:
		for itr_val in range(1, 100):
			batch_val = val_loader.__next__()
			inp_len, out_len = batch_val[0].shape[0], batch_val[0].shape[1], batch_val[-1].shape[1]
			batch_dict_val = {
				'observed_data': batch_val[0].to(device),
				'observed_tp': torch.arange(0, inp_len).to(device) / 12,
				'data_to_predict': batch_val[-1].to(device),
				'tp_to_predict': torch.arange(inp_len, inp_len+out_len).to(device) / 12,
				'observed_mask': torch.ones(batch_val[0].shape).to(device),
				'mask_predicted_data': None,
				'labels': None,
				'mode': 'extrap'
			}
			val_res = model.compute_all_losses(batch_dict_val, n_traj_samples = 3, kl_coef = kl_coef)
			if itr_val == 1:
				loss_val = val_res["loss"]
			else:
				loss_val += val_res["loss"]

./
./
./


KeyboardInterrupt: 

In [22]:
train_res

{'loss': tensor(30.1813, device='cuda:0', grad_fn=<MeanBackward0>),
 'likelihood': tensor(-31.3523, device='cuda:0'),
 'mse': tensor(0.0070, device='cuda:0'),
 'pois_likelihood': tensor(0., device='cuda:0'),
 'ce_loss': tensor(0., device='cuda:0'),
 'kl_first_p': tensor(3.2290, device='cuda:0'),
 'std_first_p': tensor(0.0311, device='cuda:0')}

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

batch = val_loader.__next__()
inp_len, out_len = batch[0].shape[1], batch[-1].shape[1]
batch_dict = {
    'observed_data': batch[0].to(device),
    'observed_tp': torch.arange(0, inp_len).to(device) / 12,
    'data_to_predict': batch[-1].to(device),
    'tp_to_predict': torch.arange(inp_len, inp_len+out_len).to(device) / 12,
    'observed_mask': torch.ones(batch[0].shape).to(device),
    'mask_predicted_data': None,
    'labels': None,
    'mode': 'extrap'
}

pred_y, info = model.get_reconstruction(batch_dict["tp_to_predict"], 
                                        batch_dict["observed_data"], 
                                        batch_dict["observed_tp"], 
                                        mask = batch_dict["observed_mask"], 
                                        n_traj_samples = 100,
                                        mode = batch_dict["mode"])
pred_y = pred_y.detach().cpu().numpy()
y = batch_dict['data_to_predict'].detach().cpu().numpy()



In [20]:
pred_y.shape[1]

32

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset_train, 
                                            batch_size=32,
                                            shuffle=True,
                                            drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset_val,
                                         batch_size=32,
                                         shuffle=True,
                                         drop_last=True)

In [ ]:
# 'observed_data', 'observed_tp', 'data_to_predict', 'tp_to_predict', 'observed_mask', 'mask_predicted_data', 'labels', 'mode'
device = 'cuda'
for batch in train_loader:
    batch_dict = {
        'observed_data': batch[0].to(device),
        'observed_tp': torch.arange(0, batch[0].shape[1]).unsqueeze(0).repeat(batch[0].shape[0], 1).to(device) / 12,
        'data_to_predict': batch[-1].to(device),
        'tp_to_predict': torch.arange(batch[0].shape[1], batch[0].shape[1] + batch[-1].shape[1]).unsqueeze(0).repeat(batch[-1].shape[0], 1).to(device) / 12,
        'observed_mask': torch.ones(batch[0].shape).to(device),
        'mask_predicted_data': None,
        'labels': None,
        'mode': 'extrap'
    }
    break

In [7]:
class args_parser:
    dataset = 'periodic'
    n = 100
    niters = 300
    lr = 1e-2
    batch_size = 50
    viz = True
    save = 'experiments/'
    load = None
    random_seed = 1991
    sample_tp = None
    cut_tp = None
    quantization = 0.1
    latent_ode = True
    z0_encoder = 'odernn'
    latents = 10
    rec_dims = 20
    rec_layers = 1
    gen_layers = 1
    units = 100
    gru_units = 100
    
    extrap = True    
    poisson = False
    classif = False
    linear_classif = False
    
    timepoints = 100
    max_t = 5.
    noise_weight = 0.01

args = args_parser()

In [8]:
# Create data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_obj = parse_datasets(args, device)

input_dim = data_obj["input_dim"]
classif_per_tp = False
n_labels = 1

In [12]:
batch_dict = latentode_utils.get_next_batch(data_obj["train_dataloader"])


In [15]:
batch_dict['observed_tp'].shape

torch.Size([50])

In [ ]:
# Create the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

obsrv_std = 0.01
obsrv_std = torch.Tensor([obsrv_std]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

model = create_LatentODE_model(args, 1, z0_prior, obsrv_std, device, 
                                classif_per_tp = False,
                                n_labels = 1)

In [ ]:
ckpt_path = '/home/grads/m/mrsergazinov/latent_ode/experiments/experiment_30327.ckpt'
utils.get_ckpt_model(ckpt_path, model, device)

In [ ]:
batch_dict = utils.get_next_batch(data_obj["train_dataloader"])

In [ ]:
batch_dict.keys()

In [ ]:
batch_dict['observed_data'].shape 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

batch_dict = utils.get_next_batch(data_obj["train_dataloader"])

pred_y, info = model.get_reconstruction(batch_dict["tp_to_predict"], 
                                        batch_dict["observed_data"], 
                                        batch_dict["observed_tp"], 
                                        mask = batch_dict["observed_mask"], 
                                        n_traj_samples = 100,
                                        mode = batch_dict["mode"])
pred_y = pred_y.detach().cpu().numpy()
y = batch_dict['data_to_predict'].detach().cpu().numpy()

fig, axs = plt.subplots(5, pred_y.shape[1] // 5, figsize=(20, 10))
for i in range(pred_y.shape[1]):
    ax = axs[i % 5, i // 5]
    df = pd.DataFrame(pred_y[:, i, :, 0].transpose(1, 0))
    df = pd.melt(df.reset_index(), id_vars=['index'], value_vars=df.columns)
    df['type'] = 'pred'
    df2 = pd.DataFrame(y[i, :, 0])
    df2 = pd.melt(df2.reset_index(), id_vars=['index'], value_vars=df2.columns)
    df2['type'] = 'true'
    df = pd.concat([df, df2])
    sns.lineplot(x="index", y="value", hue='type', data=df, ax=ax)
fig.savefig("pred_vs_true.png")

# df_pred_y_batch0 = pd.DataFrame(pred_y[:, 0, :, 0].transpose(1, 0))
# df_pred_y_batch0 = pd.melt(df_pred_y_batch0.reset_index(), id_vars=['index'], value_vars=df_pred_y_batch0.columns)

# # plot the data
# fig, ax = plt.subplots(figsize=(10, 6))
# sns.lineplot(x="index", y="value", data=df_pred_y_batch0, ax=ax)
# # save the figure
# fig.savefig("pred_y_batch0.png")

# # plot batch_dict['data_to_predict'][0, :, 0].detach().cpu().numpy() and save
# df_y_batch0 = pd.DataFrame()
# df_y_batch0["value"] = batch_dict['data_to_predict'][0, :, 0].detach().cpu().numpy()
# df_y_batch0["index"] = np.arange(df_y_batch0.shape[0])
# fig, ax = plt.subplots(figsize=(10, 6))
# sns.lineplot(x="index", y="value", data=df_y_batch0, ax=ax)
# fig.savefig("y_batch0.png")

In [ ]:
log_path = "logs/" + file_name + "_" + str(experimentID) + ".log"
if not os.path.exists("logs/"):
	utils.makedirs("logs/")
logger = utils.get_logger(logpath=log_path, filepath='./')

optimizer = optim.Adamax(model.parameters(), lr=args.lr)

num_batches = data_obj["n_train_batches"]

for itr in range(1, num_batches * (args.niters + 1)):
	optimizer.zero_grad()
	utils.update_learning_rate(optimizer, decay_rate = 0.999, lowest = args.lr / 10)

	wait_until_kl_inc = 10
	if itr // num_batches < wait_until_kl_inc:
		kl_coef = 0.
	else:
		kl_coef = (1-0.99** (itr // num_batches - wait_until_kl_inc))

	batch_dict = utils.get_next_batch(data_obj["train_dataloader"])
	train_res = model.compute_all_losses(batch_dict, n_traj_samples = 3, kl_coef = kl_coef)
	train_res["loss"].backward()
	optimizer.step()

	n_iters_to_viz = 1
	if itr % (n_iters_to_viz * num_batches) == 0:
		with torch.no_grad():

			test_res = compute_loss_all_batches(model, 
				data_obj["test_dataloader"], args,
				n_batches = data_obj["n_test_batches"],
				experimentID = experimentID,
				device = device,
				n_traj_samples = 3, kl_coef = kl_coef)

			message = 'Epoch {:04d} [Test seq (cond on sampled tp)] | Loss {:.6f} | Likelihood {:.6f} | KL fp {:.4f} | FP STD {:.4f}|'.format(
				itr//num_batches, 
				test_res["loss"].detach(), test_res["likelihood"].detach(), 
				test_res["kl_first_p"], test_res["std_first_p"])
		
			logger.info("Experiment " + str(experimentID))
			logger.info(message)
			logger.info("KL coef: {}".format(kl_coef))
			logger.info("Train loss (one batch): {}".format(train_res["loss"].detach()))
			logger.info("Train CE loss (one batch): {}".format(train_res["ce_loss"].detach()))
			
			if "auc" in test_res:
				logger.info("Classification AUC (TEST): {:.4f}".format(test_res["auc"]))

			if "mse" in test_res:
				logger.info("Test MSE: {:.4f}".format(test_res["mse"]))

			if "accuracy" in train_res:
				logger.info("Classification accuracy (TRAIN): {:.4f}".format(train_res["accuracy"]))

			if "accuracy" in test_res:
				logger.info("Classification accuracy (TEST): {:.4f}".format(test_res["accuracy"]))

			if "pois_likelihood" in test_res:
				logger.info("Poisson likelihood: {}".format(test_res["pois_likelihood"]))

			if "ce_loss" in test_res:
				logger.info("CE loss: {}".format(test_res["ce_loss"]))

		torch.save({
			'args': args,
			'state_dict': model.state_dict(),
		}, ckpt_path)